### Midterm project - Kaggle exercise project

https://www.kaggle.com/c/house-prices-advanced-regression-techniques

Goals

* Get familar with Kaggle competition since our final project may be a real Kaggle competition.
* Practice on large data set
* Data quality checking, missing data imputation
* Feature selection. What's your rationals to choose those features.
* Create at least 5 new features and explain your reason for every new features.
* Use feature importance to help you choose 10 features
* Choose 3 ML algorithms we covered in the class to build your prediction models
* For each model you build, you will need to evaluate and show your effort to improve it.
* Your jupyter submission should be in an article quality. Do NOT print huge data set in the notebook. Use head(). Use plotting to visualize your analysis and results. Use markdown to write your comments.
* Your score is based on completeness on every step.
* Submit in HTML and ipynb format on canvas

In [243]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [244]:
train_data = pd.read_csv('train.csv')
train_data.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [245]:
#selecting features from existing train data
df_feat = train_data[['LotArea', 'YearBuilt', 'Neighborhood','OverallQual',
                              'OverallCond','HouseStyle','CentralAir','BsmtQual', 'GrLivArea', 
                              'TotRmsAbvGrd', 'GarageType','YearRemodAdd','SaleCondition','YrSold']]

df_feat.head()

,LotArea,YearBuilt,Neighborhood,OverallQual,OverallCond,HouseStyle,CentralAir,BsmtQual,GrLivArea,TotRmsAbvGrd,GarageType,YearRemodAdd,SaleCondition,YrSold
0,8450,2003,CollgCr,7,5,2Story,Y,Gd,1710,8,Attchd,2003,Normal,2008
1,9600,1976,Veenker,6,8,1Story,Y,Gd,1262,6,Attchd,1976,Normal,2007
2,11250,2001,CollgCr,7,5,2Story,Y,Gd,1786,6,Attchd,2002,Normal,2008
3,9550,1915,Crawfor,7,5,2Story,Y,TA,1717,7,Detchd,1970,Abnorml,2006
4,14260,2000,NoRidge,8,5,2Story,Y,Gd,2198,9,Attchd,2000,Normal,2008


In [246]:
#shows which columns have missing data
df_feat.isna().any()

LotArea          False
YearBuilt        False
Neighborhood     False
OverallQual      False
OverallCond      False
HouseStyle       False
CentralAir       False
BsmtQual          True
GrLivArea        False
TotRmsAbvGrd     False
GarageType        True
YearRemodAdd     False
SaleCondition    False
YrSold           False
dtype: bool

In [247]:
#replacing NA for no basement and no garage with None before label encoding

df_feat['BsmtQual'].fillna('None',  inplace=True,)
df_feat['GarageType'].fillna('None',  inplace=True)

In [248]:
#shows that none of the data columns are NA anymore
df_feat.isna().any()

LotArea          False
YearBuilt        False
Neighborhood     False
OverallQual      False
OverallCond      False
HouseStyle       False
CentralAir       False
BsmtQual         False
GrLivArea        False
TotRmsAbvGrd     False
GarageType       False
YearRemodAdd     False
SaleCondition    False
YrSold           False
dtype: bool

In [249]:
#creating the 5 new features and adding to df_feat dataframe

df_feat['FinishBsmtSF'] = train_data['TotalBsmtSF'] - train_data['BsmtUnfSF']

df_feat['YearsSinceRemodel'] = train_data['YrSold'] - train_data['YearRemodAdd']

df_feat['TotalLivArea'] = train_data['GrLivArea'] + df_feat['FinishBsmtSF'] 

df_feat['TotalRooms'] = train_data['BsmtFullBath'] + train_data['BsmtHalfBath'] + train_data['FullBath'] + train_data['HalfBath']+ train_data['BedroomAbvGr']+ train_data['KitchenAbvGr']
df_feat.head()

,LotArea,YearBuilt,Neighborhood,OverallQual,OverallCond,HouseStyle,CentralAir,BsmtQual,GrLivArea,TotRmsAbvGrd,GarageType,YearRemodAdd,SaleCondition,YrSold,FinishBsmtSF,YearsSinceRemodel,TotalLivArea,TotalRooms
0,8450,2003,CollgCr,7,5,2Story,Y,Gd,1710,8,Attchd,2003,Normal,2008,706,5,2416,8
1,9600,1976,Veenker,6,8,1Story,Y,Gd,1262,6,Attchd,1976,Normal,2007,978,31,2240,7
2,11250,2001,CollgCr,7,5,2Story,Y,Gd,1786,6,Attchd,2002,Normal,2008,486,6,2272,8
3,9550,1915,Crawfor,7,5,2Story,Y,TA,1717,7,Detchd,1970,Abnorml,2006,216,36,1933,6
4,14260,2000,NoRidge,8,5,2Story,Y,Gd,2198,9,Attchd,2000,Normal,2008,655,8,2853,9


In [250]:
#encoding the categorical features (Neighborhood, HouseStyle, GarageType, Sale Condition)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


# generate binary values using get_dummies
copy_df = pd.get_dummies(df_feat["Neighborhood"], columns=['Neighborhood'], prefix='Neigh')
df_feat = df_feat.join(copy_df)

copy_df = pd.get_dummies(df_feat["HouseStyle"], columns=['HouseStyle'],  prefix='Style')
df_feat = df_feat.join(copy_df)

copy_df = pd.get_dummies(df_feat["GarageType"], columns=['GarageType'],  prefix='Garage')
df_feat = df_feat.join(copy_df)

copy_df = pd.get_dummies(df_feat["SaleCondition"], columns=['SaleCondition'],  prefix='SaleCond')
df_feat = df_feat.join(copy_df)

copy_df = pd.get_dummies(df_feat["CentralAir"], columns=['CentralAir'], prefix='CentralAir')
df_feat = df_feat.join(copy_df)

copy_df = pd.get_dummies(df_feat["BsmtQual"], columns=['BsmtQual'], prefix='BsmtQual')
df_feat = df_feat.join(copy_df)


df_feat.head() 

,LotArea,YearBuilt,Neighborhood,OverallQual,OverallCond,HouseStyle,CentralAir,BsmtQual,GrLivArea,TotRmsAbvGrd,...,SaleCond_Family,SaleCond_Normal,SaleCond_Partial,CentralAir_N,CentralAir_Y,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA
0,8450,2003,CollgCr,7,5,2Story,Y,Gd,1710,8,...,0,1,0,0,1,0,0,1,0,0
1,9600,1976,Veenker,6,8,1Story,Y,Gd,1262,6,...,0,1,0,0,1,0,0,1,0,0
2,11250,2001,CollgCr,7,5,2Story,Y,Gd,1786,6,...,0,1,0,0,1,0,0,1,0,0
3,9550,1915,Crawfor,7,5,2Story,Y,TA,1717,7,...,0,0,0,0,1,0,0,0,0,1
4,14260,2000,NoRidge,8,5,2Story,Y,Gd,2198,9,...,0,1,0,0,1,0,0,1,0,0


In [251]:

df = df_feat.copy()
df = df.drop("Neighborhood",1 )
df = df.drop("HouseStyle",1)
df = df.drop("GarageType",1)
df = df.drop("SaleCondition",1)
df = df.drop("CentralAir",1)
df = df.drop("BsmtQual",1)

df.head()

,LotArea,YearBuilt,OverallQual,OverallCond,GrLivArea,TotRmsAbvGrd,YearRemodAdd,YrSold,FinishBsmtSF,YearsSinceRemodel,...,SaleCond_Family,SaleCond_Normal,SaleCond_Partial,CentralAir_N,CentralAir_Y,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA
0,8450,2003,7,5,1710,8,2003,2008,706,5,...,0,1,0,0,1,0,0,1,0,0
1,9600,1976,6,8,1262,6,1976,2007,978,31,...,0,1,0,0,1,0,0,1,0,0
2,11250,2001,7,5,1786,6,2002,2008,486,6,...,0,1,0,0,1,0,0,1,0,0
3,9550,1915,7,5,1717,7,1970,2006,216,36,...,0,0,0,0,1,0,0,0,0,1
4,14260,2000,8,5,2198,9,2000,2008,655,8,...,0,1,0,0,1,0,0,1,0,0


In [258]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
model = LinearRegression()
#sc = StandardScaler()
X_train = df
#X_train = sc.fit_transform(X_train)
y_train = train_data['SalePrice']
model.fit(X_train,y_train)

coeff_parameter = pd.DataFrame(model.coef_,df.columns,columns=['Coefficient'])
print(coeff_parameter[0:30])
print(coeff_parameter[30:65])

                    Coefficient
LotArea                0.569008
YearBuilt            271.573528
OverallQual        13331.079514
OverallCond         6495.050437
GrLivArea             31.987077
TotRmsAbvGrd        2326.228496
YearRemodAdd          -6.839013
YrSold               -81.013795
FinishBsmtSF          -8.495512
YearsSinceRemodel    -74.174785
TotalLivArea          23.491584
TotalRooms            87.968702
Neigh_Blmngtn      -8570.982939
Neigh_Blueste     -20446.656002
Neigh_BrDale      -22089.176620
Neigh_BrkSide       -768.470491
Neigh_ClearCr       5022.913460
Neigh_CollgCr       3487.586963
Neigh_Crawfor      15783.585073
Neigh_Edwards     -21066.370990
Neigh_Gilbert       -419.217406
Neigh_IDOTRR      -10439.349630
Neigh_MeadowV     -23245.325880
Neigh_Mitchel     -17870.527029
Neigh_NAmes       -12605.598206
Neigh_NPkVill     -12171.527673
Neigh_NWAmes      -12259.895954
Neigh_NoRidge      61604.816816
Neigh_NridgHt      37913.121706
Neigh_OldTown     -17017.496639
        

In [253]:
test_data = pd.read_csv('test.csv')
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
